# Subgraph (함수 래핑): 상태 변환하기

이 노트북에서는 **함수로 서브그래프를 래핑**하여 **상태 키가 달라도 사용**할 수 있는 방법을 배웁니다.

## 함수 래핑이 필요한 이유

```
┌────────────────────────────────────────────────────────────────────┐
│                    상태 키가 다른 경우                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   부모 State:                     서브그래프 State:                │
│   ┌─────────────┐                ┌─────────────┐                  │
│   │ foo: str    │                │ bar: str    │  ← 키가 다름!    │
│   └─────────────┘                │ baz: str    │                  │
│                                  └─────────────┘                  │
│                                                                    │
│   직접 연결 불가! → 함수로 래핑하여 상태 변환 필요                 │
│                                                                    │
│   foo → bar (입력 변환)                                            │
│   bar → foo (출력 변환)                                            │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    함수 래핑 방식                                   │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   부모 그래프                                                       │
│   ┌──────────────────────────────────────────┐                     │
│   │                                          │                     │
│   │   START → ┌─────────────────────┐ → END  │                     │
│   │           │      node 함수      │        │                     │
│   │           │                     │        │                     │
│   │           │  foo → bar (변환)   │        │                     │
│   │           │  ┌───────────────┐  │        │                     │
│   │           │  │  서브그래프   │  │        │                     │
│   │           │  │   invoke()    │  │        │                     │
│   │           │  └───────────────┘  │        │                     │
│   │           │  bar → foo (변환)   │        │                     │
│   │           │                     │        │                     │
│   │           └─────────────────────┘        │                     │
│   │                                          │                     │
│   └──────────────────────────────────────────┘                     │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langgraph

# 2. State 정의 (키가 다름)

In [ ]:
from typing import TypedDict

# 부모 그래프의 상태
class State(TypedDict):
    foo: str  # 부모 그래프의 키

# 서브그래프의 상태 (부모와 키가 다름!)
class SubgraphState(TypedDict):
    bar: str  # 서브그래프의 키 (foo와 다름)
    baz: str

print("✅ State 정의 완료")
print("   - State (부모): foo")
print("   - SubgraphState: bar, baz (부모와 공유 키 없음!)")

# 3. 서브그래프 정의

In [ ]:
from langgraph.graph import StateGraph, START

def subgraph_node(state: SubgraphState):
    """
    서브그래프의 노드
    
    bar 값에 'baz' 추가
    """
    print(f"  [서브그래프] 입력 bar: {state['bar']}")
    
    new_bar = state['bar'] + 'baz'
    
    print(f"  [서브그래프] 출력 bar: {new_bar}")
    return {'bar': new_bar}


# 서브그래프 빌더
subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node('subgraph_node', subgraph_node)
subgraph_builder.add_edge(START, 'subgraph_node')

# 서브그래프 컴파일
subgraph = subgraph_builder.compile()

print("✅ 서브그래프 정의 완료")

# 4. 래퍼 함수 정의 (핵심!)

부모 상태와 서브그래프 상태 간의 변환을 담당합니다.

In [ ]:
def node(state: State):
    """
    서브그래프를 호출하는 래퍼 노드
    
    1. 부모 상태 → 서브그래프 상태 변환
    2. 서브그래프 호출
    3. 서브그래프 결과 → 부모 상태 변환
    """
    print(f"[래퍼] 부모 상태 foo: {state['foo']}")
    
    # 1. 부모 상태 → 서브그래프 상태 변환
    subgraph_input = {'bar': state['foo']}  # foo → bar
    print(f"[래퍼] 변환: foo → bar")
    
    # 2. 서브그래프 호출
    response = subgraph.invoke(subgraph_input)
    
    # 3. 서브그래프 결과 → 부모 상태 변환
    print(f"[래퍼] 변환: bar → foo")
    return {'foo': response['bar']}  # bar → foo


print("✅ 래퍼 함수 정의 완료")

# 5. 부모 그래프 정의

In [ ]:
# 부모 그래프 빌더
builder = StateGraph(State)

# 래퍼 함수를 노드로 추가 (서브그래프가 아닌 함수!)
builder.add_node('node', node)  # 래퍼 함수 추가
builder.add_edge(START, 'node')

# 부모 그래프 컴파일
graph = builder.compile()

print("✅ 부모 그래프 정의 완료")
print("   래퍼 함수가 'node' 노드로 추가됨")

In [ ]:
# 그래프 구조 시각화
print("=== 부모 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 6. 그래프 실행

In [ ]:
# 부모 그래프 실행
initial_state = {'foo': 'hello'}

print("=== 부모 그래프 실행 ===")
print(f"입력: {initial_state}")
print()

result = graph.invoke(initial_state)

print()
print(f"출력: {result}")
print()
print("흐름: foo('hello') → bar('hello') → bar('hellobaz') → foo('hellobaz')")

# 7. 복잡한 변환 예시

In [ ]:
# 더 복잡한 변환이 필요한 경우
class ParentState(TypedDict):
    user_input: str
    result: str

class ChildState(TypedDict):
    query: str
    answer: str

# 자식 서브그래프 노드
def child_node(state: ChildState):
    return {'answer': f"처리됨: {state['query']}"}

child_builder = StateGraph(ChildState)
child_builder.add_node('process', child_node)
child_builder.add_edge(START, 'process')
child_graph = child_builder.compile()

# 복잡한 변환 래퍼
def complex_wrapper(state: ParentState):
    # 입력 변환: user_input → query
    child_input = {
        'query': state['user_input'].upper(),  # 대문자로 변환
        'answer': ''
    }
    
    # 서브그래프 호출
    response = child_graph.invoke(child_input)
    
    # 출력 변환: answer → result (추가 가공)
    return {'result': f"[완료] {response['answer']}"}

# 부모 그래프
parent_builder = StateGraph(ParentState)
parent_builder.add_node('wrapper', complex_wrapper)
parent_builder.add_edge(START, 'wrapper')
parent_graph = parent_builder.compile()

print("✅ 복잡한 변환 예시 준비 완료")

In [ ]:
# 복잡한 변환 테스트
print("=== 복잡한 변환 테스트 ===")
result = parent_graph.invoke({'user_input': 'hello world', 'result': ''})

print(f"입력: user_input='hello world'")
print(f"출력: result='{result['result']}'")
print()
print("변환 흐름:")
print("  1. user_input='hello world'")
print("  2. query='HELLO WORLD' (대문자 변환)")
print("  3. answer='처리됨: HELLO WORLD'")
print("  4. result='[완료] 처리됨: HELLO WORLD'")

---

## 정리: Subgraph 함수 래핑

### 핵심 개념

```
┌─────────────────────────────────────────────────────────────────────┐
│                    함수 래핑 패턴                                   │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   def wrapper_node(parent_state: ParentState):                      │
│       # 1. 입력 변환                                                │
│       child_input = convert_to_child(parent_state)                  │
│                                                                     │
│       # 2. 서브그래프 호출                                          │
│       response = subgraph.invoke(child_input)                       │
│                                                                     │
│       # 3. 출력 변환                                                │
│       return convert_to_parent(response)                            │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 래퍼 함수
def node(state: State):
    # foo → bar 변환
    response = subgraph.invoke({'bar': state['foo']})
    # bar → foo 변환
    return {'foo': response['bar']}

# 부모 그래프에서 래퍼 함수를 노드로 추가
builder.add_node('node', node)  # 서브그래프가 아닌 함수!
```

### 직접 연결 vs 함수 래핑 비교

| 방식 | 상태 공유 | 변환 가능 | 복잡도 | 유연성 |
|------|----------|----------|--------|--------|
| **직접 연결** | 필수 | 불가능 | 낮음 | 낮음 |
| **함수 래핑** | 불필요 | 가능 | 높음 | 높음 |

### 사용 시나리오

| 시나리오 | 권장 방식 |
|----------|----------|
| 같은 상태 구조 | 직접 연결 |
| 다른 상태 구조 | 함수 래핑 |
| 변환/가공 필요 | 함수 래핑 |
| 재사용 서브그래프 | 함수 래핑 |

## 다음 단계

**Supervisor 패턴**을 사용하여 여러 에이전트를 관리하는 방법을 배웁니다. (04-05번 노트북)